In [1]:
!pip install -U pymgrid

In [3]:
import numpy as np
from pymgrid import Microgrid
from pymgrid.modules import BatteryModule, LoadModule, RenewableModule, GridModule

battery = BatteryModule(min_capacity=0,
                        max_capacity=100,
                        max_charge=50,
                        max_discharge=50,
                        efficiency=1.0,
                        init_soc=0.5)

# Using random data
renewable = RenewableModule(time_series=50*np.random.rand(100))

load = LoadModule(time_series=60*np.random.rand(100))

grid = GridModule(max_import = 100, max_export=100, time_series=np.random.rand(100,3))

microgrid = Microgrid([battery, ("pv", renewable), load, grid])

In [4]:
microgrid

Microgrid([load x 1, pv x 1, balancing x 1, battery x 1, grid x 1])

In [5]:
microgrid.fixed

{
  "load": "[LoadModule(time_series=<class 'numpy.ndarray'>, forecaster=NoForecaster, forecast_horizon=0, forecaster_increase_uncertainty=False, raise_errors=False)]"
}

In [6]:
microgrid.flex

{
  "pv": "[RenewableModule(time_series=<class 'numpy.ndarray'>, raise_errors=False, forecaster=NoForecaster, forecast_horizon=0, forecaster_increase_uncertainty=False, provided_energy_name=renewable_used)]",
  "balancing": "[UnbalancedEnergyModule(raise_errors=False, loss_load_cost=10.0, overgeneration_cost=2.0)]"
}

In [7]:
microgrid.controllable

{
  "battery": "[BatteryModule(min_capacity=0, max_capacity=100, max_charge=50, max_discharge=50, efficiency=1.0, battery_cost_cycle=0.0, battery_transition_model=None, init_charge=None, init_soc=0.5, raise_errors=False)]",
  "grid": "[GridModule(max_import=100, max_export=100)]"
}

In [8]:
empty_action=microgrid.get_empty_action
print(empty_action)

<bound method Microgrid.get_empty_action of Microgrid([load x 1, pv x 1, balancing x 1, battery x 1, grid x 1])>


In [10]:
microgrid.reset()
microgrid.state_series.to_frame()

0
load    0 load_current         -39.997473
pv      0 renewable_current     14.306967
battery 0 soc                    0.500000
          current_charge        50.000000
grid    0 import_price_current   0.996086
          export_price_current   0.769397
          co2_per_kwh_current    0.573774
          grid_status_current    1.000000

In [11]:
load = -1.0 * microgrid.modules.load.item().current_load
pv = microgrid.modules.pv.item().current_renewable

In [12]:
net_load = load + pv

if net_load > 0:
    net_load = 0.0

In [13]:
battery_discharge = min(-1*net_load, microgrid.modules.battery.item().max_production)
net_load += battery_discharge

In [14]:
grid_import = min(-1*net_load, microgrid.modules.grid.item().max_production)

In [15]:
control = {"battery" : [battery_discharge],
           "grid": [grid_import]}

control

{'battery': [25.690506262325325], 'grid': [-0.0]}

In [21]:
for i in range(10):
  load = -1.0 * microgrid.modules.load.item().current_load
  pv = microgrid.modules.pv.item().current_renewable
  net_load = load + pv

  if net_load > 0:
    net_load = 0.0

  battery_discharge = min(-1*net_load, microgrid.modules.battery.item().max_production)
  net_load += battery_discharge

  grid_import = min(-1*net_load, microgrid.modules.grid.item().max_production)
  control = {"battery" : [battery_discharge],
           "grid": [grid_import]}
  microgrid.run(control)

In [22]:
microgrid.get_log()

module_name     load                             pv              \
module_number      0                              0               
field         reward   load_met load_current reward curtailment   
0                0.0  39.997473   -39.997473    0.0    0.000000   
1                0.0   6.354509    -6.354509    0.0    0.000000   
2                0.0   7.853697    -7.853697    0.0    0.000000   
3                0.0  19.318836   -19.318836    0.0    0.000000   
4                0.0  39.693860   -39.693860    0.0    0.000000   
5                0.0  50.790374   -50.790374    0.0    0.000000   
6                0.0  33.195441   -33.195441    0.0    0.000000   
7                0.0  51.267149   -51.267149    0.0    0.000000   
8                0.0  23.090269   -23.090269    0.0   19.605876   
9                0.0  19.007274   -19.007274    0.0    0.000000   
10               0.0  21.255881   -21.255881    0.0    0.000000   

module_name                                       balancing              \
module_number                                             0               
field         renewable_used renewable_current       reward   loss_load   
0                  14.306967         14.306967    -0.000000    0.000000   
1                  11.342573         11.342573 -1450.119364  145.011936   
2                  27.565738         27.565738 -1059.784648  105.978465   
3                  35.973448         35.973448  -833.453879   83.345388   
4                  21.155323         21.155323  -685.385372   68.538537   
5                  49.038210         49.038210  -517.521636   51.752164   
6                  34.241487         34.241487 -1489.539538  148.953954   
7                  24.046595         24.046595  -772.205542   77.220554   
8                   0.000000         19.605876   -53.819463    0.000000   
9                  17.158901         17.158901  -518.483730   51.848373   
10                 36.452485         36.452485 -1348.033952  134.803395   

module_name                   ...                 grid                      \
module_number                 ...                    0                       
field         overgeneration  ... export_price_current co2_per_kwh_current   
0                   0.000000  ...             0.769397            0.573774   
1                   0.000000  ...             0.699834            0.661168   
2                   0.000000  ...             0.792299            0.518717   
3                   0.000000  ...             0.788187            0.411569   
4                   0.000000  ...             0.181629            0.321319   
5                   0.000000  ...             0.186904            0.417291   
6                   0.000000  ...             0.236600            0.916832   
7                   0.000000  ...             0.091296            0.463653   
8                  26.909731  ...             0.313669            0.047340   
9                   0.000000  ...             0.095530            0.238250   
10                  0.000000  ...             0.894978            0.043223   

module_name                            balance                                \
module_number                                0                                 
field         grid_status_current       reward overall_provided_to_microgrid   
0                             1.0     0.000000                     39.997473   
1                             1.0 -1380.135957                    156.354509   
2                             1.0  -980.554718                    133.544203   
3                             1.0  -754.635162                    119.318836   
4                             1.0  -667.222488                    139.693860   
5                             1.0  -498.831261                    150.790374   
6                             1.0 -1465.879556                    183.195441   
7                             1.0  -763.075908                    151.267149   
8                             1.0  -1

In [45]:
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Box, Discrete
from pymgrid import Microgrid
from pymgrid.modules import BatteryModule, LoadModule, RenewableModule, GridModule

In [65]:
# Microgrid setup
battery = BatteryModule(min_capacity=0,
                        max_capacity=100,
                        max_charge=50,
                        max_discharge=50,
                        efficiency=1.0,
                        init_soc=0.5)

load = LoadModule(time_series=np.random.rand(100) * 60)
renewable = RenewableModule(time_series=np.random.rand(100) * 50)
microgrid = Microgrid([battery, load, ("pv", renewable)])

class MicrogridGymEnv(Env):
    def __init__(self, microgrid):
        super(MicrogridGymEnv, self).__init__()
        self.microgrid = microgrid
        self.max_steps = len(microgrid.modules.load.item().time_series)
        self.step_count = 0

        # State: [battery SoC, load, renewable] (continuous)
        self.observation_space = Box(low=np.array([0, 0, 0]), high=np.array([100, 100, 100]), dtype=np.float32)
        # Actions: 0 (discharge), 1 (do nothing), 2 (charge)
        self.action_space = Discrete(3)

    def reset(self, seed=None, **kwargs):
        # Handle seeding for reproducibility
        if seed is not None:
            self._np_random = np.random.RandomState(seed)
            np.random.seed(seed)

        self.microgrid.reset()
        self.step_count = 0
        state = self._get_state()
        return state, {}

    def step(self, action_idx):
        action_map = {0: {"battery": [-20]}, 1: {"battery": [0]}, 2: {"battery": [20]}}
        action = action_map[action_idx]
        observation, reward, done, info =self.microgrid.run(action)
        self.step_count += 1

        if not done:
            state = self._get_state()
        else:
            # Return the previous state or a default state if needed
            state = self.observation_space.sample()  # Or some other suitable value
        return state, reward, done, False, {}

    def _get_state(self):
        return np.array([microgrid.modules.battery.item().max_production,
                         self.microgrid.modules.load.item().current_load,
                         microgrid.modules.pv.item().current_renewable], dtype=np.float32)

env = MicrogridGymEnv(microgrid)

In [36]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [69]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env

# Verify environment compatibility
check_env(env)  # Raises errors if the env doesn’t meet Gym standards

# Initialize DQN model
model = DQN(
    policy="MlpPolicy",  # Multi-layer perceptron for Q-network
    env=env,
    learning_rate=0.001,
    buffer_size=10000,
    batch_size=32,
    gamma=0.99,  # Discount factor
    exploration_fraction=0.1,  # Fraction of training time for exploration
    exploration_initial_eps=1.0,
    exploration_final_eps=0.01,
    target_update_interval=10,  # Update target network every 10 steps
    verbose=1
)

# Train the model
model.learn(total_timesteps=10000)  # Adjust timesteps based on your needs

# Save the model
model.save("dqn_microgrid")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 99        |
|    ep_rew_mean      | -2.03e+04 |
|    exploration_rate | 0.608     |
| time/               |           |
|    episodes         | 4         |
|    fps              | 357       |
|    time_elapsed     | 1         |
|    total_timesteps  | 396       |
| train/              |           |
|    learning_rate    | 0.001     |
|    loss             | 271       |
|    n_updates        | 73        |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 99        |
|    ep_rew_mean      | -2.02e+04 |
|    exploration_rate | 0.216     |
| time/               |           |
|    episodes         | 8         |
|    fps              | 341       |
|    time_elapsed     | 2         |
|    total_timesteps  | 792       |
| trai

In [67]:
microgrid.get_log()

module_name     load                             pv              \
module_number      0                              0               
field         reward   load_met load_current reward curtailment   
0                0.0  42.911362   -42.911362    0.0   13.500399   

module_name                                     balancing            \
module_number                                           0             
field         renewable_used renewable_current     reward loss_load   
0                        0.0         13.500399 -14.177276       0.0   

module_name                   ...       battery                      \
module_number                 ...             0                       
field         overgeneration  ... charge_amount  soc current_charge   
0                   7.088638  ...           0.0  0.5           50.0   

module_name      balance                                \
module_number          0                                 
field             reward overall_provided_to_microgrid   
0             -14.177276                          50.0   

module_name                                    \
module_number                                   
field         overall_absorbed_from_microgrid   
0                                        50.0   

module_name                                       \
module_number                                      
field         controllable_provided_to_microgrid   
0                                           50.0   

module_name                                         \
module_number                                        
field         controllable_absorbed_from_microgrid   
0                                              0.0   

module_name                                                              
module_number                                                            
field         fixed_provided_to_microgrid fixed_absorbed_from_microgrid  
0                                     0.0                     42.911362  

[1 rows x 22 columns]